In [ ]:
import pandas as pd
import os
import re
import numpy as np
import cv2
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
from zipfile import ZipFile
import shutil
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Sensor Data

In [ ]:
def read_data(data):

    data.reset_index(inplace = True)
    data.rename(columns={'level_0': 'Time'}, inplace = True)
    data.rename(columns={'x-axis (deg/s).4': 'Raw Brainwave Signal '}, inplace = True)
    data.rename(columns={'Unnamed: 42': 'Tag'}, inplace = True)

    TimeStamp = data.iloc[:,0]
    ankle = data.iloc[: , 1:8]
    pocket = data.iloc[:, 8:15]
    waist = data.iloc[:,15:22]
    neck = data.iloc[:,22:29]
    wrist = data.iloc[:,29:36]
    EEG = data.iloc[:,36]
    Infraded = data.iloc[:,37:43]
    label = data.iloc[:,46]

    ankle.columns = ['X-axis Accelerometer (g)', 'Y-axis Accelerometer (g)' , 'Z-axis Accelerometer (g)',
                     'Roll Gyroscrope (deg/s)', 'Pitch Gyroscope (deg/s)', 'Yaw Gyroscope (deg/s)' ,'Luminosity (lux)']

    pocket.columns = ['X-axis Accelerometer (g)', 'Y-axis Accelerometer (g)' , 'Z-axis Accelerometer (g)',
                     'Roll Gyroscrope (deg/s)', 'Pitch Gyroscope (deg/s)', 'Yaw Gyroscope (deg/s)' ,'Luminosity (lux)']

    waist.columns = ['X-axis Accelerometer (g)', 'Y-axis Accelerometer (g)' , 'Z-axis Accelerometer (g)',
                     'Roll Gyroscrope (deg/s)', 'Pitch Gyroscope (deg/s)', 'Yaw Gyroscope (deg/s)' ,'Luminosity (lux)']

    neck.columns = ['X-axis Accelerometer (g)', 'Y-axis Accelerometer (g)' , 'Z-axis Accelerometer (g)',
                     'Roll Gyroscrope (deg/s)', 'Pitch Gyroscope (deg/s)', 'Yaw Gyroscope (deg/s)' ,'Luminosity (lux)']

    wrist.columns = ['X-axis Accelerometer (g)', 'Y-axis Accelerometer (g)' , 'Z-axis Accelerometer (g)',
                     'Roll Gyroscrope (deg/s)', 'Pitch Gyroscope (deg/s)', 'Yaw Gyroscope (deg/s)' ,'Luminosity (lux)']

    Infraded.columns = ['Infrared 1', 'Infrared 2', 'Infrared 3', 'Infrared 4', 'Infrared 5', 'Infrared 6']

    handled_data = pd.concat([TimeStamp, ankle,pocket,waist,neck,wrist,EEG,Infraded ],
                            axis = 1,
                            keys = ['TimeStamp','Wearable Ankle', 'Wearable Pocket','Wearable Waist',
                                    'Wearable Neck', 'Wearable Wrist','EEG Headset'  ,'Infrared'],

                            names = ['Deviece Name', 'Channel Name'])
    handled_data[('Tag' , 'Label')]= label

    return handled_data


def concat_data():
    concat_Sub = []
    list_Sub = []
    sum_shape = 0
    sum_csv = 0
    for sub_ in range(1,17 + 1) :
        Sub = 'Subject' + str(sub_)
        list_Sub.append(Sub)
        concat_Act = []
        list_Act = []
        for act_ in range(1,11+1):
            Act = 'Activity'+ str(act_)
            list_Act.append(Act)
            concat_Trial = []
            list_Trial  = []
            for trial_ in range(1,3+1):
                Trial = 'Trial'+ str(trial_)
                list_Trial.append(Trial)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue
                else :
                    path = "/content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv"
                    #path = Sub + '/' + Act + '/' + Trial + '/' + Sub + Act + Trial + '.csv'
                data = pd.read_csv(path,skiprows=1)
                print('path : {} . Shape : ({},{})'.format(path, data.shape[0], data.shape[1]))
                sum_shape += data.shape[0]
                sum_csv +=1
                handled  = read_data(data)
                concat_Trial.append(handled)
            TRIAL = pd.concat(concat_Trial,keys = list_Trial)
            concat_Act.append(TRIAL)
        ACT = pd.concat(concat_Act, keys = list_Act)
        concat_Sub.append(ACT)
    SUB = pd.concat(concat_Sub,keys = list_Sub)
    return SUB


In [ ]:
SUB = concat_data()

path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1.csv . Shape : (195,43)
path : /co

In [ ]:
SUB.to_csv(path_or_buf='/content/drive/MyDrive/Sensor + CSV/Sensor.csv', index = False)

In [ ]:
times = SUB.iloc[:,0].values
labels = SUB.iloc[:,-1].values
Time_Label = pd.DataFrame(labels , index = times)
Time_Label

,0
2018-07-04T12:04:17.738369,7
2018-07-04T12:04:17.790509,7
2018-07-04T12:04:17.836632,7
2018-07-04T12:04:17.885262,7
2018-07-04T12:04:17.945423,7
...,...
2018-07-04T12:04:27.430041,11
2018-07-04T12:04:27.484688,11
2018-07-04T12:04:27.525797,11
2018-07-04T12:04:27.570416,11


## Load image

In [ ]:
def load_img(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam , DesiredWidth = 64, DesiredHeight = 64 ):
    IMG = []
    count = 0
    name_img = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)

        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):
                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue
                for cam_ in range(start_cam , end_cam + 1) :
                    cam = 'Camera'+ str(cam_)

                    with ZipFile('/content/drive/MyDrive/Sensor + CSV/Subject1Activity1Trial1Camera1.zip', 'r') as zipObj:
                        zipObj.extractall('CAMERA/' + sub + act  + trial  + cam)

                    for root, dirnames, filenames in os.walk('CAMERA/' + sub + act + trial  +cam ):
                        for filename in filenames:
                            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                                filepath = os.path.join(root, filename)
                                count += 1
                                if count % 5000 == 0 :
                                    print('{} : {} ' .format(filepath,count))
                                if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png' :
                                    print('----------------------------NO SHAPE---------------------------------')
                                    continue
                                elif len(filepath) > 70 :
                                    print(' {} : Invalid image'.format(filepath))
                                    continue
                                name_img.append(filepath)
                                img = cv2.imread(filepath, 0)
                                resized = ResizeImage(img, DesiredWidth, DesiredHeight)
                                IMG.append(resized)
                    shutil.rmtree('CAMERA/'+ sub + act + trial + cam)
    return IMG , name_img


def handle_name(path_name) :
    img_name = []
    for path in path_name :
        if len(path) == 68:
            img_name.append(path[38:64])
        elif len(path) == 69 :
            img_name.append(path[39:65])
        else :
            img_name.append(path[40:66])
    handle = []
    for name in img_name :
        n1 = 13
        a1 = name.replace(name[n1],':')
        n2 = 16
        a2 = a1.replace(name[n2],':')
        handle.append(a2)
    return handle


def ShowImage(ImageList, nRows = 1, nCols = 2, WidthSpace = 0.00, HeightSpace = 0.00):
    from matplotlib import pyplot as plt
    import matplotlib.gridspec as gridspec
    gs = gridspec.GridSpec(nRows, nCols)
    gs.update(wspace=WidthSpace, hspace=HeightSpace) # set the spacing between axes.
    plt.figure(figsize=(20,20))
    for i in range(len(ImageList)):
        ax1 = plt.subplot(gs[i])
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.set_aspect('equal')
        plt.subplot(nRows, nCols,i+1)
        image = ImageList[i].copy()
        if (len(image.shape) < 3):
            plt.imshow(image, plt.cm.gray)
        else:
            plt.imshow(image)
        plt.title("Image " + str(i))
        plt.axis('off')
    plt.show()


def ResizeImage(IM, DesiredWidth, DesiredHeight):
    OrigWidth = float(IM.shape[1])
    OrigHeight = float(IM.shape[0])
    Width = DesiredWidth
    Height = DesiredHeight

    if((Width == 0) & (Height == 0)):
        return IM

    if(Width == 0):
        Width = int((OrigWidth * Height)/OrigHeight)

    if(Height == 0):
        Height = int((OrigHeight * Width)/OrigWidth)

    dim = (Width, Height)
    resizedIM = cv2.resize(IM, dim, interpolation = cv2.INTER_NEAREST)
    return resizedIM

In [ ]:
start_sub = 1
end_sub  = 2
start_act = 1
end_act = 2
start_cam = 1
end_cam = 1
DesiredWidth = 32
DesiredHeight = 32


img_1, path_1 = load_img(start_sub ,   end_sub,
             start_act , end_act  ,
             start_cam ,   end_cam , DesiredWidth ,  DesiredHeight )


name_1 = handle_name(path_1)

In [ ]:
size = '32x32'
cam = '1'
image = 'Sensor + Image' + '/' + 'image_' + cam +  '.npy'
# name = 'Camera + Label' + '/' + size + '/' + 'name_' + cam + '(' + size + ')' + '.npy'
name = 'Sensor + Image' + '/' + 'name_' + cam +  '.npy'

name_1 = handle_name(path_1)

np.save(image, img_1)
np.save(name, name_1)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
start_sub = 1
end_sub  = 17
start_act = 1
end_act = 11
start_cam = 2
end_cam = 2

DesiredWidth = 32
DesiredHeight = 32

img_2, path_2 = load_img(start_sub ,   end_sub,
             start_act , end_act  ,
             start_cam ,   end_cam , DesiredWidth, DesiredHeight )

CAMERA/Subject1Activity6Trial2Camera2/2018-07-04T12_12_01.821800.png : 5000 
CAMERA/Subject1Activity8Trial1Camera2/2018-07-04T12_17_22.832033.png : 10000 
CAMERA/Subject1Activity11Trial1Camera2/2018-07-04T12_24_58.704636.png : 15000 
CAMERA/Subject2Activity4Trial3Camera2/2018-07-11T13_43_26.285138.png : 20000 
CAMERA/Subject2Activity7Trial2Camera2/2018-07-11T13_50_50.004135.png : 25000 
CAMERA/Subject2Activity9Trial1Camera2/2018-07-11T13_56_53.362931.png : 30000 
CAMERA/Subject3Activity1Trial2Camera2/2018-07-05T12_06_01.688633.png : 35000 
CAMERA/Subject3Activity6Trial3Camera2/2018-07-05T12_13_33.475186.png : 40000 
CAMERA/Subject3Activity8Trial1Camera2/2018-07-05T12_18_49.128592.png : 45000 
CAMERA/Subject3Activity11Trial1Camera2/2018-07-05T12_26_39.647046.png : 50000 
 CAMERA/Subject4Activity1Trial1Camera2/Subject4Activity1Trial1Camera2Synced/2018-07-10T13_27_20.4508017.png : Invalid image
 CAMERA/Subject4Activity1Trial1Camera2/Subject4Activity1Trial1Camera2Synced/2018-07-10T13_27_20

In [ ]:
print(len(img_1))
print(len(name_1))
print(len(img_2))
print(len(name_2))

294678
294678
294678
294678


In [ ]:
ind1 = np.arange(0,294678)
red_in1 = ind1[~np.isin(name_1,name_2)]

name_1d =  np.delete(name_1, red_in1[0])
img_1d = np.delete(img_1, red_in1[0], axis = 0)

In [ ]:
ind2 = np.arange(0,294678)
red_in2 = ind2[~np.isin(name_2,name_1)]

name_2d =  np.delete(name_2, red_in2[0])
img_2d = np.delete(img_2, red_in2[0], axis = 0)

In [ ]:
(name_1d == name_2d).all()

True

In [ ]:
label_1 = Time_Label.loc[name_1d].values
label_2 = Time_Label.loc[name_2d].values

In [ ]:
print(len(img_1d))
print(len(name_1d))
print(len(label_1))
print(len(img_2d))
print(len(name_2d))
print(len(label_2))

294677
294677
294677
294677
294677
294677


In [ ]:

cam = '1'

image = 'Sensor + Image' + '/' + 'image_' + cam +  '.npy'
name = 'Sensor + Image' + '/' + 'name_' + cam +  '.npy'
label = 'Sensor + Image' + '/' + 'label_' + cam +  '.npy'


np.save(image, img_1d)
np.save(name, name_1d)
np.save(label, label_1)


cam = '2'
image = 'Sensor + Image' + '/' + 'image_' + cam +  '.npy'
name = 'Sensor + Image' + '/' + 'name_' + cam +  '.npy'
label = 'Sensor + Image' + '/' + 'label_' + cam +  '.npy'

np.save(image, img_2d)
np.save(name, name_2d)
np.save(label, label_2)